In [ ]:
from dataclasses import dataclass
import opt_note.scsp as scsp
import datetime
import didppy

In [ ]:
import marimo as mo
import nbformat

# `LA_SH` アルゴリズムのサブルーチンを DIDP で置き換えてみる

In [ ]:
def find_next_strategy(
    instance: list[str],
    chars: str,
    m: int,
) -> str:
    """
    現在の状態を受け取り, m 手進めたときに sum height が最大になる文字の選び方 (長さ m 以下の文字列として表される) を返す
    """

    dpmodel = didppy.Model(maximize=True, float_cost=False)

    instance_table = dpmodel.add_element_table(
        [[chars.index(c) for c in s] + [len(chars)] for s in instance]
    )

    index_types = [dpmodel.add_object_type(number=len(s) + 1) for s in instance]
    index_vars = [
        dpmodel.add_element_var(object_type=index_type, target=0)
        for index_type in index_types
    ]
    sol_len = dpmodel.add_int_var(target=0)

    dpmodel.add_base_case([sol_len == m])

    # 文字 char に従って進む
    for id_char, char in enumerate(chars):
        sum_height = sum(
            (instance_table[sidx, index_var] == id_char).if_then_else(1, 0)
            for sidx, index_var in enumerate(index_vars)
        )
        trans = didppy.Transition(
            name=f"{char}",
            cost=sum_height + didppy.IntExpr.state_cost(),
            effects=[
                (
                    index_var,
                    (instance_table[sidx, index_var] == id_char).if_then_else(
                        index_var + 1, index_var
                    ),
                )
                for sidx, index_var in enumerate(index_vars)
            ]
            + [(sol_len, sol_len + 1)],
            preconditions=[sum_height > 0],
        )
        dpmodel.add_transition(trans)

    # Force transition
    end = didppy.Transition(
        name="",
        cost=didppy.IntExpr.state_cost(),
        effects=[(sol_len, m)],
        preconditions=[
            index_var == len(s) for s, index_var in zip(instance, index_vars)
        ],
    )
    dpmodel.add_transition(end, forced=True)

    # Dual bound
    dual_bound_table = dpmodel.add_int_table(
        [[min(m, len(s) - idx) for idx in range(len(s) + 1)] for s in instance]
    )
    bound = didppy.IntExpr(0)
    for sidx, index_var in enumerate(index_vars):
        bound += dual_bound_table[sidx, index_var]
    dpmodel.add_dual_bound(bound)

    dpsolver = didppy.CABS(dpmodel, threads=8, quiet=True)
    solution = dpsolver.search()

    return "".join([trans.name for trans in solution.transitions])

## 計算時間の比較

In [ ]:
@dataclass
class Model:
    instance: list[str]
    solution: str | None = None
    best_bound: float = 0.0

    def solve(self, m: int = 3, ll: int = 1, *args, **kwargs) -> str | None:
        chars = "".join(sorted(list(set("".join(self.instance)))))
        state = tuple(0 for _ in self.instance)
        solution = ""

        count = 0
        while not all(idx == len(s) for idx, s in zip(state, self.instance)):
            next_str = find_next_strategy(
                [s[idx:] for idx, s in zip(state, self.instance)], chars, m
            )
            if len(next_str) == 0:
                break
            count += 1
            # print(f"{count=}, {next_str=}")
            solution += next_str[:ll]
            for next_char in next_str[:ll]:
                state = tuple(
                    idx + 1 if idx < len(s) and s[idx] == next_char else idx
                    for idx, s in zip(state, self.instance)
                )

        self.solution = solution
        return self.solution

In [ ]:
scsp.util.bench(
    scsp.model.la_sh.Model, example_filename="uniform_q26n004k015-025.txt"
)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 70) ---
 Sol: itkgnekojiqfouhlcimpnbvaxxnhtqcgxycozgbrddbcsovxiorsuqzpplpvinngfssbxf
str1: -tkgn-k------uh---mp----x-nhtq-gx---z---------vxi--s------------------
str2: i------ojiqfo--l----nb--xx----c---------------v----suq-p---vi----ssbxf
str3: -------------u-lci--n------------yco--------sov--o----zpplp-----------
str4: i--g-e----------------va------------zgbrddbcs-v---r--------v-nngf-----

example file name: 'uniform_q26n004k015-025.txt'
best objective: 70
best bound: 0.0
wall time: 0.0081s


In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n004k015-025.txt")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 68) ---
 Sol: itkgnekuhlcojimpxnhtqfycoslnbgxzoxcvaozxigpsubqrddbcspvirssvnblngxpf
str1: -tkgn-kuh-----mpxnhtq--------gxz---v---xi--s------------------------
str2: i----------oji------qf--o-lnb-x--xcv-------su-q------pvi-ss--b---x-f
str3: -------u-lc--i---n----ycos------o--v-oz---p----------p--------l---p-
str4: i--g-e-----------------------------va-z--g---b-rddbcs-v-r--vn--ng--f

example file name: 'uniform_q26n004k015-025.txt'
best objective: 68
best bound: 0.0
wall time: 8.158647s


In [ ]:
scsp.util.bench(
    scsp.model.la_sh.Model, example_filename="uniform_q26n008k015-025.txt"
)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 132) ---
 Sol: eingeojipbdcevaqdcrtkzfgjtbnkuhlcimpvxdnerddbycopflrhnzbsmorvigoplersbrowtxqgxckrdrlctodtmzprplpucpmqvxbbghinngfstdfuiqpvcdissbowdxf
str1: -------------------tk--g---nkuh---mp-x-n------------h--------------------t-qgx------------z----------vx----i----s-------------------
str2: -i---oji-------q------f------------------------o--l--n-b------------------x--xc----------------------v----------s---u-qpv--issb---xf
str3: -----------------------------u-lci-----n-----yco--------s-o-v--o--------------------------zp-plp------------------------------------
str4: -i-ge--------va------z-g--b--------------rddb-c---------s---v------r---------------------------------v------nngf--------------------
str5: 

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n008k015-025.txt")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 112) ---
 Sol: igepnbdcevazfojtdcrvxkgberydpfzdbulcinrsvzwxqkruvnfychmoplxsnbdromqvhiglctodztmprplqgxxcfuizvcwdsuqxpvbisseobhxf
str1: ---------------t-----kg--------------n-------k-u-----hm-p-x-n-------h----t---------qgx-----zv------x---is-------
str2: i------------oj---------------------i-------q-----f----o-l--nb-----------------------xxc----v---suq-pv-iss--b-xf
str3: ---------------------------------ulcin-------------yc--o---s----o--v------o-z--p-pl-----------------p-----------
str4: ige------vaz----------gb-r-d---db--c---sv-----r-vn----------n---------g-----------------f-----------------------
str5: ---p----------------------y-p-----l---r--z-x---u----c---p--------mqv--g--t-d------------fui-vc-ds---

In [ ]:
scsp.util.bench(
    scsp.model.la_sh.Model, example_filename="uniform_q26n016k015-025.txt"
)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 170) ---
  Sol: kpriflshtkqagenbkvxuwdgsfchxeqjafnvzfmpacdigqbcejtivwbahcokrdxerlcnvpfzksgjmiudbcrsvihtjodqbgkderwxzavntfmoybplngqrbpzvxucbbhefikoshmgopmqvgtowdfuivxcdvsbouqyzpplpvissbxf
str01: --------tk--g-n-k--u------h----------mp----------------------x----n------------------ht---q-g-----xz-v-----------------x-------i--s---------------------------------------
str02: ---i-----------------------------------------------------o----------------j-i-------------q-------------f-o---ln---b--

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q26n016k015-025.txt")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 156) ---
  Sol: kripxwuslxqhftkcgevaxjnfzpabdieusvdcgbqarvdnypfzhckodjiqbcesfmolvrzwxiuchozvnbpmgqjpwxanlctvfodkxgtdohmfkpskerzgqbcrtoqvigpxszluivcwdsjbqopvxissaekgnbhwwxfy
str01: -------------tk-g-----n---------------------------k-------------------u-h------m---p-x-n-------------h--------------t-q--g-x-z---v----------xis-------------
str02: --i------------------------------------------------o-jiq----f-ol------------nb-------x----------x-----------------c----v----s--u--------q-pv-iss--

In [ ]:
scsp.util.bench(
    scsp.model.la_sh.Model, example_filename="uniform_q05n010k010-010.txt"
)

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: abdcbacdbeecdaebdecbadabcdebdde
str01: --dcb-c----cd--b--c-----c-e----
str02: -bd----dbee---e--e-b-d---------
str03: ---c-acd-eec--eb-e-------------
str04: a--------e--d---d----d---debdd-
str05: a--cb----eec-a-b--c-------e----
str06: -b--ba--be-----bd-cba----------
str07: -b--ba---e---aeb----ada--------
str08: ---------ee---e--ecb-d-b--e---e
str09: ---c--cd-ee-da--d-c--d---------
str10: -bd--a--b---d--b-e--a-a--d-----

example file name: 'uniform_q05n010k010-010.txt'
best objective: 31
best bound: 0.0
wall time: 0.004362s


In [ ]:
scsp.util.bench(Model, example_filename="uniform_q05n010k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: bdacbacdbeecdaeebadcbddeacbddee
str01: -d-cb-c----cd---b--c-----c---e-
str02: bd-----dbee---eeb-d------------
str03: ---c-acd-eec--e-b------e-------
str04: --a------e--d-----d--dde--bdd--
str05: --acb----eec-a--b--c---e-------
str06: b---ba--be------b-dcb---a------
str07: b---ba---e---ae-bad-----a------
str08: ---------ee---ee---cbd----b--ee
str09: ---c--cd-ee-da----dc-d---------
str10: bda-b--dbe---a---ad------------

example file name: 'uniform_q05n010k010-010.txt'
best objective: 31
best bound: 0.0
wall time: 0.175682s


In [ ]:
scsp.util.bench(
    scsp.model.la_sh.Model, example_filename="uniform_q05n050k010-010.txt"
)

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: dacbedabcedabcdeaecbdacebad

In [ ]:
scsp.util.bench(Model, example_filename="uniform_q05n050k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: daecbdaebdacbecadebacbdebac

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="nucleotide_n010k010.txt")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 28) ---
  Sol: TATACACGGGATACGAATCGATCACAGA
str01: -AT----GGGATACG-------------
str02: -ATAC-C----T-----TC---C-C---
str03: ----CACG--A-A----T---T----GA
str04: TA-A-A----AT-C---T-G-T------
str05: -A-----GG--TA--A--C-A--A-A--
str06: T-T-C-C----TA-G----G-T-A----
str07: T-T----G---TA-GA-TC--T------
str08: T------GGGA-A-G--T---TC-----
str09: T-T-C-C---A--C-AA-C--T------
str10: T---C------TA--AA-CGA--A----

example file name: 'nucleotide_n010k010.txt'
best objective: 28
best bound: 0.0
wall time: 0.002583s


In [ ]:
scsp.util.bench(Model, example_filename="nucleotide_n010k010.txt")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 27) ---
  Sol: TATCACGTAGGATACTGAATCCTAAAC
str01: -AT---G--GGATAC-G----------
str02: -AT-AC--------CT---TCC----C
str03: ---CACG-A--AT--TGA---------
str04: TA--A---A--AT-CTG--T-------
str05: -A----G--G--TA---A--C--AAA-
str06: T-TC-C-TAGG-TA-------------
str07: T-T---GTAG-AT-CT-----------
str08: T-----G--GGA-A--G--T--T---C
str09: T-TC-C--A-----C--AA-C-T----
str10: T--C---TA--A-AC-GAA--------

example file name: 'nucleotide_n010k010.txt'
best objective: 27
best bound: 0.0
wall time: 0.171892s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="nucleotide_n050k050.txt")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
scsp.util.bench(Model, example_filename="nucleotide_n050k050.txt")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="protein_n010k010.txt")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MEQSKAFPLSVEYACHPGFLRVNTAEQHFRDAIKGGRALHDNPSRNGTVYQ
str01: M----A--LS--Y-C-P----------------KG------------T---
str02: M-QS-----S---------L--N-A-------I---------P-----V--
str03: M------PLS--Y-------------QHFR---K-----------------
str04: ME---------E---H-----VN--E------------LHD----------
str05: M--S------------------N-----F-DAI---RAL------------
str06: M-----F-------------R-N---Q--------------N-SRNG----
str07: M-----F-----YA-H--------A---F-----GG-------------Y-
str08: M--SK-F----------------T-----R------R-----P------YQ
str09: M--S--F---V--A---G---V-TA-Q------------------------
str10: ME-S----L-V-----PGF---N--E-------------------------

example file name: 'protein_n010k010.txt'
best objective: 51
best bound: 0.0
wall time: 0.034957s


In [ ]:
scsp.util.bench(Model, example_filename="protein_n010k010.txt")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 51) ---
  Sol: MESKFALVPLSYAGQEHVFTRNAEQNSRFSLNCPKDAYQIGGHDPRALTVY
str01: M----AL---SY--------------------CPK-----G-------T--
str02: M-------------Q-----------S--SLN----A--I----P----V-
str03: M-------PLSY--Q-H-F-R-------------K----------------
str04: ME-------------EHV---N-E------L-----------HD-------
str05: M-S------------------N------F------DA--I-----RAL---
str06: M---F---------------RN--QNSR---N--------G----------
str07: M---F------YA---H-----A-----F-----------GG--------Y
str08: M-SKF--------------TR------R-----P---YQ------------
str09: M-S-F--V----AG---V-T--A-Q--------------------------
str10: MES---LVP----G----F--N-E---------------------------

example file name: 'protein_n010k010.txt'
best objective: 51
best bound: 0.0
wall time: 0.899211s


In [ ]:
scsp.util.bench(scsp.model.la_sh.Model, example_filename="protein_n050k050.txt")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

In [ ]:
scsp.util.bench(Model, example_filename="protein_n050k050.txt")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

DIDP を使うと時間が長くなる.
小さな問題であれば直接プログラミングした方がよい.

## 先読みの手数を増やせないか

In [ ]:
@dataclass
class Model2:
    instance: list[str]
    solution: str | None = None
    best_bound: float = 0.0

    def solve(self, m: int = 5, ll: int = 1, *args, **kwargs) -> str | None:
        model = Model(self.instance)
        self.solution = model.solve(m=m, ll=ll)
        self.best_bound = model.best_bound
        return self.solution

In [ ]:
scsp.util.bench(Model2, example_filename="uniform_q26n004k015-025.txt")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 67) ---
 Sol: tkulcignyceojiqfsovaozppglkuhmpxnbhtqgxzxcvrddxbicsuqpvirssbvnngfxf
str1: tk----gn------------------kuhmpxn-htqgxz--v---x-i-s----------------
str2: -----i-----ojiqf-o-------l------nb----x-xcv-------suqpvi-ssb-----xf
str3: --ulci-nyc-o----sov-ozpp-l----p------------------------------------
str4: -----ig---e-------va-z--g--------b---------rdd-b-cs---v-r---vnngf--

example file name: 'uniform_q26n004k015-025.txt'
best objective: 67
best bound: 0.0
wall time: 0.686045s


In [ ]:
scsp.util.bench(Model2, example_filename="uniform_q26n008k015-025.txt")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 105) ---
 Sol: igpbdevyplrazxudnglbcvrdpzfojimtwxqkvgrtdforlnbycxzostkouivhdtmperxcvdzsbruoqznpvinhtqgssbplbpxhzvxieswdf
str1: -------------------------------t---k-g-------n--------k-u--h--mp--x-----------n----htqg-------x-zvxi-s---
str2: i--------------------------oji----q------fo-lnb--x----------------xcv--s--u-q--pvi-----ssb----x---------f
str3: --------------u---l-c--------i---------------n-yc--os--o--v----------------o-z-p----------pl-p-----------
str4: ig---ev----az----g-b--rd----------------d-----b-c---s-----v------r--v---------n---n---g-----------------f
str5: --p----yplr-zxu-----c---p-----m---q-vg-tdf--------------uiv--------c-d-sb--o-----------------------------
str6: --pbdev--------d----cv-

In [ ]:
scsp.util.bench(Model2, example_filename="uniform_q26n016k015-025.txt")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 142) ---
  Sol: krixwushltxxkqjgenfpvabcfzdigkusiqevbrhnydcovmdpeflrzjtavxebniucsvdqwhakorvntfpohmlngqszbgxzrovqxicgtdpvfsuqpliokkvecdistskmpwrbplxfgijobhwdwy
str01: ---------t--k--g-n-----------ku-------h------m-p---------x--n--------h------t--------q---gxz--v-xi-------s------------------------------------
str02: --i----------------------------------------o---------j-------i-----q---------f-o--ln----b-x-----x-c----v-suqp-----v---is-s-----b--xf----------
str03: -----u--l--------------c

In [ ]:
scsp.util.bench(Model2, example_filename="uniform_q05n010k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 31) ---
  Sol: bdbacbeacdebeedcbaedbcdceeadbdd
str01: -d--cb--c------c---dbc-ce------
str02: bd-------d-bee----e-----e---bd-
str03: ----c--acde-e--c--e-b---e------
str04: ---a--e--d----d----d--d-e---bdd
str05: ---acbe---e----c-a--bc--e------
str06: b-ba-be----b--dcba-------------
str07: b-ba--ea--eb-----a-d------a----
str08: ------e---e-ee-cb--db---ee-----
str09: ----c---cde-e-d--a-d-cd--------
str10: bd-a-b---d-be----a--------ad---

example file name: 'uniform_q05n010k010-010.txt'
best objective: 31
best bound: 0.0
wall time: 0.216772s


In [ ]:
scsp.util.bench(Model2, example_filename="uniform_q05n050k010-010.txt")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 38) ---
  Sol: dacebcdaebdacdebcaebdacebad

In [ ]:
scsp.util.bench(Model2, example_filename="nucleotide_n010k010.txt")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 26) ---
  Sol: TCATACGGCTAGATACTGTAAATCCT
str01: --AT--GG---GATAC-G--------
str02: --ATAC--CT---T-C-------CC-
str03: -CA--CG---A-AT--TG-A------
str04: T-A-A-----A-AT-CTGT-------
str05: --A---GG-TA-A--C---AAA----
str06: T--T-C--CTAG-----GTA------
str07: T--T--G--TAGAT-CT---------
str08: T-----GG---GA-A--GT---TC--
str09: T--T-C--C-A----C---AA--C-T
str10: TC-TA-----A-A--C-G-AA-----

example file name: 'nucleotide_n010k010.txt'
best objective: 26
best bound: 0.0
wall time: 0.171989s


In [ ]:
scsp.util.bench(Model2, example_filename="nucleotide_n050k050.txt")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
scsp.util.bench(Model2, example_filename="protein_n010k010.txt")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 47) ---
  Sol: MQEPASFLSLRNYVACPKQGEHVFTDANSIRARPEFGNGKVLHDQYQ
str01: M---A--LS---Y--CPK-G----T----------------------
str02: MQ---S--SL-N--A--------------I---P------V------
str03: M--P---LS---Y-----Q--H-F------R--------K-------
str04: M-E-----------------EHV----N------E------LHD---
str05: M----S-----N-----------F-DA--IRA---------L-----
str06: M-----F---RN------Q--------NS-R------NG--------
str07: M-----F-----Y-A------H----A--------FG-G------Y-
str08: M----S-----------K-----FT-----R-RP-----------YQ
str09: M----SF------VA----G--V-T-A-----------------Q--
str10: M-E--S-L-----V--P--G---F---N------E------------

example file name: 'protein_n010k010.txt'
best objective: 47
best bound: 0.0
wall time: 1.060496s


In [ ]:
# 5 分以内に計算が終わらなかったのでスキップ

# scsp.util.bench(Model2, example_filename="protein_n050k050.txt")

改善はしたが, 計算時間の増加に見合うほどではなかった.